In [ ]:
import cv2
import math
import time
import queue
import threading
from PIL import Image

from utils.thresholding import *
from pynq import Overlay
from pynq import MMIO
import time
import matplotlib.pyplot as plt
%matplotlib inline
from pynq import allocate
import pynq.lib.dma
import psutil

In [ ]:
# Load overlay for lane navigation and controlling motors
overlay = Overlay("lane_detection.bit")

In [ ]:
# Helper to set bits
def set_bit(value, bit):
    return value | (1 << bit)

In [ ]:
# Helper to extract timer registers
def extract_timer_regs(timer_name):
    regs = overlay.ip_dict[timer_name]['registers']
    return {
        'TCSR0_addr': regs['TCSR0']['address_offset'],
        'TCSR1_addr': regs['TCSR1']['address_offset'],
        'TCSR0_fields': regs['TCSR0']['fields'],
        'TCSR1_fields': regs['TCSR1']['fields'],
        'TLR0_addr': regs['TLR0']['address_offset'],
        'TLR1_addr': regs['TLR1']['address_offset'],
    }

In [ ]:
# Configure control register to enable timer in PWM mode
def build_control_value(fields, reg_fields):
    val = 0
    for field in fields:
        val = set_bit(val, reg_fields[field]['bit_offset'])
    return val

In [ ]:
# Command motor rotation direction
def control_motors(motor_en, l_forward, r_forward):
    gpio_val = 0

    # Set left side motors' bits
    gpio_val |= 0x4 if l_forward else 0x8

    # Set right side motors' bits
    gpio_val |= 0x1 if r_forward else 0x2

    # Write combined value once
    motor_en.write(0x0, gpio_val)

In [ ]:
def get_crop_indices(points, high, low):
    bottom_crop, top_crop = 0, len(points)
    for i in range(len(points)):
        if points[i] >= high:
            top_crop = i
            break
    
    for i in range(len(points) - 1, -1, -1):
        if points[i] <= low:
            bottom_crop = i + 1
            break
    
    return top_crop, bottom_crop

In [ ]:
def compute_lane_heading_angle(left_fit=None, right_fit=None, y_eval=200):
    angles = []

    def angle_from_fit(fit):
        # Derivative of the 2nd-order polynomial at y = y_eval
        dy = 1.0  # pixel change in y
        dx = 2 * fit[0] * y_eval + fit[1]  # derivative at y_eval
        angle_rad = np.arctan(dx)  # slope to angle
        return -np.degrees(angle_rad)

    if left_fit is not None:
        angles.append(angle_from_fit(left_fit))
    if right_fit is not None:
        angles.append(angle_from_fit(right_fit))

    if angles:
        return np.mean(angles)
    else:
        return None

In [ ]:
def detect_lane_lines_connected_components(binary_warped):
    # Ensure the input is binary (0 and 255)
    binary = np.uint8(binary_warped * 255) if binary_warped.max() <= 1 else np.uint8(binary_warped)

    # Apply connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)

    # Image dimensions
    height, width = binary.shape

    # Store points
    leftx, lefty, rightx, righty = [], [], [], []

    for i in range(1, num_labels):  # Label 0 is background
        x, y, w, h, area = stats[i]
        cx, cy = centroids[i]

        # Heuristic filters for likely lane lines
        if area > 1000 and h > 30:
            if cx < width // 2:
                coords = np.column_stack(np.where(labels == i))
                for pt in coords:
                    lefty.append(pt[0])
                    leftx.append(pt[1])
            else:
                coords = np.column_stack(np.where(labels == i))
                for pt in coords:
                    righty.append(pt[0])
                    rightx.append(pt[1])

    # Convert lists to numpy arrays
    leftx = np.array(leftx)
    lefty = np.array(lefty)
    rightx = np.array(rightx)
    righty = np.array(righty)

    return leftx, lefty, rightx, righty

In [ ]:
# Target dimensions for downsizing resolution
RESIZE_W = 320
RESIZE_H  = 180

# Needs comments
M1 = allocate(shape=(RESIZE_H * RESIZE_W * 3), dtype=np.int32)
M2 = allocate(shape=(20002), dtype=np.int32)

crop_height = int(RESIZE_H  * 0.5)

frame_in_w = 640
frame_in_h = 480

ym_per_pix = 0.1524 / 72.0
xm_per_pix = 0.2286 / 600.0
y_eval = 175 #Point on y axis for angle evaluation

angle = 0
kernel = np.ones((3, 3), np.uint8)  # Needs comments

In [ ]:
def capture_frames(videoIn, frame_queue):
    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to grab frame.")
            break
        if frame_queue.full():
            # If the queue is full (only 1 frame), remove the oldest frame
            frame_queue.get()  # Discard the old frame to keep the queue size at 1
        frame_queue.put(frame)

In [ ]:
# PWM SETUP (Using MMIO).....
right_motor = overlay.axi_timer_0 #PMODB P3 for right PWM
left_motor = overlay.axi_timer_1  #PMODB P5 for left PWM

# Extract register addresses (will be the same for every Axi Timer)
R_TCSR0 = overlay.ip_dict['axi_timer_0']['registers']['TCSR0']
R_TCSR1 = overlay.ip_dict['axi_timer_0']['registers']['TCSR1']
R_TCSR0_address = R_TCSR0['address_offset']
R_TCSR1_address = R_TCSR1['address_offset']
R_TCSR0_register = R_TCSR0['fields'] # bit_offset for address
R_TCSR1_register = R_TCSR1['fields']
R_TLR0 = overlay.ip_dict['axi_timer_0']['registers']['TLR0']
R_TLR1 = overlay.ip_dict['axi_timer_0']['registers']['TLR1']
R_TLR0_address = R_TLR0['address_offset']
R_TLR1_address = R_TLR1['address_offset']

# Create the configuration values for the control register
R_temp_val_0 = 0
R_temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
R__temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['PWMA0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['GENT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['UDT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['UDT1']['bit_offset'])

# Set Autoreload (ARHT0 = 1)
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['ARHT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['ARHT1']['bit_offset'])

# Enable timer (ENT0 = 1)
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['ENT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['ENT1']['bit_offset'])

In [ ]:
L_TCSR0 = overlay.ip_dict['axi_timer_1']['registers']['TCSR0']
L_TCSR1 = overlay.ip_dict['axi_timer_1']['registers']['TCSR1']
L_TCSR0_address = L_TCSR0['address_offset']
L_TCSR1_address = L_TCSR1['address_offset']
L_TCSR0_register = L_TCSR0['fields'] # bit_offset for address
L_TCSR1_register = L_TCSR1['fields']
L_TLR0 = overlay.ip_dict['axi_timer_1']['registers']['TLR0']
L_TLR1 = overlay.ip_dict['axi_timer_1']['registers']['TLR1']
L_TLR0_address = L_TLR0['address_offset']
L_TLR1_address = L_TLR1['address_offset']

# Create the configuration values for the control register
L_temp_val_0 = 0
L_temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['PWMA0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['GENT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['UDT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['UDT1']['bit_offset'])

# Set Autoreload (ARHT0 = 1)
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['ARHT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['ARHT1']['bit_offset'])

# Enable timer (ENT0 = 1)
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['ENT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['ENT1']['bit_offset']) 

In [ ]:
# All these... needs comments
CONTROL_REGISTER = 0x0
lane_detection_ip = overlay.lane_detection_0
lane_detection_ip.write(CONTROL_REGISTER, 0x00)  # Reset
lane_detection_ip.write(CONTROL_REGISTER, 0x81)  # Start
dma0 = overlay.axi_dma_0
dma0_send = overlay.axi_dma_0.sendchannel
dma0_recv = overlay.axi_dma_0.recvchannel

# GPIO SETUP
motor_enable_address = overlay.ip_dict['axi_gpio_0']['phys_addr']
motor_enable = MMIO(motor_enable_address, 8) 
motor_enable.write(0x4, 0x0) # Write to tri-state register to configure IO as OUTPUT

print("Program running")

# Check if frame retrieval from camera is possible
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
print("capture device is open: " + str(videoIn.isOpened()))

frame_queue = queue.Queue(maxsize=1)

# Start thread getting images from camera
capture_thread = threading.Thread(target=capture_frames, args=(videoIn, frame_queue))
capture_thread.daemon = True
capture_thread.start()

In [ ]:
dt = 0.01                  
duration = 10              
t = np.arange(0, duration, dt)

# PID coefficients
Kp = 10
Ki = 0.3
Kd = 0.3

# Gain for output of PID controller
total_gain = 0.1

integral = 0
prev_error = 0
base_power = 30 # Average power (baseline duty cycle)

r_power = 30
l_power = 30

original = -90
error = -90
    
l_forward = True
r_forward = True

curr_time = 0
prev_time = time.time();

_period_ = 10000000 # 50Hz, 20ms

frame_list = []
 
for i in range(1, 50): 
    # Wait until next frame arrives
    while (frame_queue.empty()):
        time.sleep(0.15)
    
    # Get image frame
    frame_vga = frame_queue.get()
    
    # Downsize image resolution (480 x 640 -> 180 x 320)
    resized_image = cv2.resize(frame_vga, dsize=(RESIZE_W, RESIZE_H), interpolation=cv2.INTER_LINEAR)
    frame_list.append(resized_image)
    
    np.copyto(M1, resized_image.flatten())

    # DMA and lane detection
    dma0_send.transfer(M1)
    dma0_recv.transfer(M2)
    dma0_send.wait()
    dma0_recv.wait()
    
    # Everything below needs comments down to where we dampen the angles
    leftx, lefty, rightx, righty = ([], [], [], [])
    left_count, right_count = M2[0], M2[1]
    left_fit, right_fit = [0,0,0], [0,0,0]
    
    leftx = M2[2:left_count + 2]
    lefty = M2[5002: 5002 + left_count]
    rightx = M2[10002:10002 + right_count]
    righty = M2[15002:15002 + right_count]
    
    left_crop_top, left_crop_bottom = get_crop_indices(lefty, 15, 180)
    right_crop_top, right_crop_bottom = get_crop_indices(righty, 15, 180)
    
    lefty, leftx = lefty[left_crop_top:left_crop_bottom], leftx[left_crop_top:left_crop_bottom]
    righty, rightx = righty[right_crop_top:right_crop_bottom], rightx[right_crop_top:right_crop_bottom]
    
    left_count, right_count = len(lefty), len(righty)

    if left_count != 0 and right_count != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        angle = compute_lane_heading_angle(left_fit=left_fit, right_fit=right_fit, y_eval=y_eval)
    elif right_count != 0:
        right_fit = np.polyfit(righty, rightx, 2)
        angle = compute_lane_heading_angle(right_fit=right_fit, y_eval=y_eval)
    elif left_count != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        angle = compute_lane_heading_angle(left_fit=left_fit, y_eval=y_eval)
                    
    # Account for high readings deep into turn due to camera's low FOV 
    if (abs(angle) >= 25):
        angle = angle*0.75
        
    print("Steering angle: " + str(angle))
    
    curr_time = time.time()
    dt = curr_time - prev_time

    error = angle
    
    #Proportional
    P = Kp * error

    #Integral
    integral += error * dt
    I = Ki * integral

    #Derivative
    derivative = error / dt
    D = Kd * derivative

    # Sum up components and apply gain
    correction = total_gain * (P + I + D)

    # Correct around average power
    r_power = base_power - correction
    l_power = base_power + correction
    
    #Keep between 0 and 100
    l_power = max(1, min(99, l_power))
    r_power = max(1, min(99, r_power))
    
    print("Correction: " + str(correction))
    print("r_power: " + str(r_power))
    print("l_power: " + str(l_power))
    
    prev_error = error
    prev_time = curr_time
    
    # Get pulse 
    period = int((_period_ & 0x0ffff) *100);
    r_pulse = int((int(r_power) & 0x07f)*period/100);
    l_pulse = int((int(l_power) & 0x07f)*period/100);
    
    # Set motor rotation direction
    control_motors(motor_enable, l_forward, r_forward)

    # Set period and duty cycle
    right_motor.write(R_TLR0_address, period)
    right_motor.write(R_TLR1_address, r_pulse)
    left_motor.write(L_TLR0_address, period)
    left_motor.write(L_TLR1_address, l_pulse)
    
    # Define the LOAD bits (1 << bit_offset)
    R_LOAD0 = 1 << R_TCSR0_register['LOAD0']['bit_offset']
    R_LOAD1 = 1 << R_TCSR1_register['LOAD1']['bit_offset']
    L_LOAD0 = 1 << L_TCSR0_register['LOAD0']['bit_offset']
    L_LOAD1 = 1 << L_TCSR1_register['LOAD1']['bit_offset']

    # Pulse LOAD bits
    right_motor.write(R_TCSR0_address, R_temp_val_0 | R_LOAD0)
    right_motor.write(R_TCSR1_address, R_temp_val_1 | R_LOAD1)
    left_motor.write(L_TCSR0_address, L_temp_val_0 | L_LOAD0)
    left_motor.write(L_TCSR1_address, L_temp_val_1 | L_LOAD1)

    # Re-enable timer (in case LOAD cleared ENT bits)
    right_motor.write(R_TCSR0_address, R_temp_val_0)
    right_motor.write(R_TCSR1_address, R_temp_val_1)
    left_motor.write(L_TCSR0_address, L_temp_val_0)
    left_motor.write(L_TCSR1_address, L_temp_val_1)

    time.sleep(0.3)  # Prevent busy loop for PWM

# Display downsized images captured from camera during run on track and their thresholded versions
for frame_vga in frame_list:
    test_image = cv2.cvtColor(frame_vga.reshape(RESIZE_H, RESIZE_W, 3).astype(np.uint8), cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10,40))
    plt.subplot(121)
    plt.imshow(test_image)
    plt.title('Original Image')
    plt.xticks([])
    plt.yticks([])

    thresholded = threshold(test_image)
    plt.subplot(1,2,2)    
    plt.imshow(thresholded, cmap='gray')
    plt.title('Thresholded Binary')